This script do:
- preprocess:
    - reset time zone to Asia/Shanghei 
    
- add features
    - new_ip by day7,8,9, assign before day7(include) as `-1`
    - potetial_fraud (ip<12620)    
- reduce features
    - use only (according model2_lgbm gain importance)
        - app_mean_target
        - app
        - channel_mean_target
        - next_click_dt
        - channel_nunique_ip
        - os
        - channel
        - hour
        - app_nunique_channel
        - channel_cnt_ip_app
        - hour_nunique_day_ip
        - app_nunique_ip
        - app_nunique_ip_device_os
        - ip        
        
compare to features1.ipynb

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import pytz
%matplotlib inline 
import gc,time

In [2]:
with pd.HDFStore('../input/raw_data.h5') as store:
    print(store.keys())
    test_df = store['test_df']
    train_df = store['train_df']

['/test_df', '/train_df']


* add highly potential fraud ip <-- ip<126420

In [72]:
train_df['potential_fraud'] = (train_df.ip<126420)

In [73]:
mask = train_df.day==7
train_df[mask].groupby('potential_fraud')['ip'].count()

potential_fraud
False    12878129
True     46831240
Name: ip, dtype: int64

In [17]:
mask = train_df.day==8 
train_df[mask].groupby('potential_fraud')['ip'].count()

potential_fraud
False    12195663
True     50165286
Name: ip, dtype: int64

In [18]:
mask = train_df.day==9
train_df[mask].groupby('potential_fraud')['ip'].count()

potential_fraud
False    11610048
True     51222594
Name: ip, dtype: int64

- add new_ip : ip is not shown in previous day

In [3]:
mask6 = train_df.day==6
ip_day6 = train_df[mask6].ip.unique()

mask7 = train_df.day==7
mask8 = train_df.day==8
mask9 = train_df.day==9
mask10 = train_df.day ==10
ip_day7 = train_df[mask7].ip.unique()
ip_day8 = train_df[mask8].ip.unique()
ip_day9 = train_df[mask9].ip.unique()

In [4]:
prev_ip_day8 = np.unique(np.append(ip_day6,ip_day7))
prev_ip_day9 = np.unique(np.append(prev_ip_day8,ip_day8))
# prev_ip_day9 = np.append(ip_day7,ip_day8)
del ip_day7,ip_day8,ip_day9;gc.collect()

0

In [11]:
train_df[mask7].tail()

,ip,app,device,os,channel,click_time,is_attributed,hour,day
59709847,40056,15,1,19,3,2017-11-07 23:59:59+08:00,0,23,7
59709848,26995,6,2,16,459,2017-11-07 23:59:59+08:00,0,23,7
59709849,157011,18,1,1,107,2017-11-07 23:59:59+08:00,0,23,7
59709850,89946,2,1,19,435,2017-11-07 23:59:59+08:00,0,23,7
59709851,48615,24,1,22,178,2017-11-07 23:59:59+08:00,0,23,7


In [10]:
newip_prev8.index[0]

59709852

In [38]:
train_df[train_df.day<=7].shape

(59709852, 9)

In [12]:
not_valid = np.full(newip_prev8.index[0],-1) ## nparray

In [5]:
newip_prev8 = ~train_df[mask8].ip.isin(prev_ip_day8)

newip_prev8.value_counts()

False    52412023
True      9948926
Name: ip, dtype: int64

In [20]:
newip_prev8 = np.array(newip_prev8).astype('uint8') ## change to numpy array

In [6]:
newip_prev9 = ~train_df[mask9].ip.isin(prev_ip_day9)

newip_prev9.value_counts()

False    56367324
True      6465318
Name: ip, dtype: int64

In [22]:
newip_prev9 = np.array(newip_prev9).astype('uint8')

In [25]:
ip_day9 = train_df[mask9].ip.unique();
prev_ip_day10 = np.unique(np.append(prev_ip_day9,ip_day9))
newip_prev10 = ~train_df[mask10].ip.isin(prev_ip_day10)


In [43]:
newip_prev9.shape

(62832642,)

In [27]:
newip_prev10 = np.array(newip_prev10).astype('uint8')

In [50]:
newip_tr = np.concatenate((not_valid,newip_prev8,newip_prev9,newip_prev10))

In [51]:
train_df['newip'] = newip_tr

In [52]:
gc.collect()

1238

In [74]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed,hour,day,newip,potential_fraud
0,83230,3,1,13,379,2017-11-06 22:32:21+08:00,0,22,6,-1,True
1,17357,3,1,19,379,2017-11-06 22:33:34+08:00,0,22,6,-1,True
2,35810,3,1,13,379,2017-11-06 22:34:12+08:00,0,22,6,-1,True
3,45745,14,1,13,478,2017-11-06 22:34:52+08:00,0,22,6,-1,True
4,161007,3,1,13,379,2017-11-06 22:35:08+08:00,0,22,6,-1,False


In [57]:
trn_ip = train_df.ip.unique()

for test newip ?

In [66]:
newip_test = ~test_df.ip.isin(trn_ip)
test_df['newip'] = newip_test.astype('uint8')

In [68]:
test_df.head()

,click_id,ip,app,device,os,channel,click_time,hour,day,newip
0,0,5744,9,1,3,107,2017-11-10 12:00:00+08:00,12,10,0
1,1,119901,9,1,3,466,2017-11-10 12:00:00+08:00,12,10,0
2,2,72287,21,1,19,128,2017-11-10 12:00:00+08:00,12,10,0
3,3,78477,15,1,13,111,2017-11-10 12:00:00+08:00,12,10,0
4,4,123080,12,1,13,328,2017-11-10 12:00:00+08:00,12,10,0


____

In [70]:
mask = (train_df.day>7) & (train_df.day<10)
trn_df = train_df[mask]

In [75]:
len_trn = len(trn_df)
trn_df = trn_df.append(test_df)

del train_df;gc.collect()

1288

In [76]:
print('trn_df :{}'.format(trn_df.shape)) # 
# print('test_df:{}'.format(test_df.shape))
del test_df

trn_df :(143984060, 11)


save temporary 

- len_trn  : 125193591 

- len_tst  : 18790469

In [86]:
store = pd.HDFStore('../input/feat/feat3_partial2.h5')
store['trn_df'] = trn_df
store.close()
    

load 

In [3]:
with pd.HDFStore('../input/feat/feat3_partial2.h5') as store:
    trn_df = store['trn_df']

# helper func

In [8]:
def encode_agg_feature(train_df, selcols, groupby, aggregator = 'nunique'):
    usecols = [e for e in selcols if e not in  groupby]    
    
    if aggregator == 'nunique':  
        colname= usecols[-1] + '_nunique_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].nunique().reset_index().\
            rename(columns = {
                usecols[-1] : usecols[-1] + '_nunique_' + '_'.join(groupby)
            })
        train_df = train_df.merge(gp, how='left', on=groupby)

    elif aggregator == 'cumcount':
        colname = usecols[-1] + '_cumcnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].cumcount()        
        train_df[colname] = gp.values
    
    elif aggregator == 'count':
        colname = usecols[-1] + '_cnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
        gp = train_df[selcols].groupby(groupby)[usecols].count().reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
        
    elif aggregator in ['var','mean']:
        agg=np.var if aggregator == 'var' else np.mean
        
        colname = usecols[-1] + '_'+ str(aggregator) +'_' + '_'.join(groupby)
        
        if colname in train_df.columns:
            train_df.drop(colname,axis=1, inplace=True)
            
        gp = train_df[selcols].groupby(groupby).agg(agg).reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
    
    return train_df.iloc[:,-1]

#### feat_base
- channel nunique by ip
- app nunique by channel
- channel cnt by (ip,app)
- hour nunique by (ip,day)
- app nunique by ip
- app nunique by (ip,device,os)

In [9]:
print('feature1 : number of unique "channel" by ip',end='\t')
f1 = encode_agg_feature(trn_df,groupby=['ip'], selcols=['ip','channel'],aggregator='nunique')
print(f1.name)
trn_df[f1.name] = f1.values.astype('uint8') ; del f1;gc.collect()

print('feature3 : number of unique "hour" by (ip, day)',end='\t')
f3 = encode_agg_feature(trn_df,selcols=['hour','ip','day'], groupby=['ip','day'], aggregator='nunique')
print(f3.name)
trn_df[f3.name] = f3.values.astype('uint8'); del f3;gc.collect()


print('feature4 : number of unique "app" by ip', end='\t')
f4 = encode_agg_feature(trn_df,selcols=['app', 'ip'], groupby=['ip'])
print(f4.name)
trn_df[f4.name] = f4.values.astype('uint8'); del f4; gc.collect()



print('feature7 : number of unique "app" by channel',end='\t')
f7 = encode_agg_feature(trn_df,selcols=['app', 'channel'], groupby=['channel'])
print(f7.name)
trn_df[f7.name] = f7.values.astype('uint8'); del f7; gc.collect()


print('feature9 : number of unique "app" by (ip,device,os)',end='\t')
f9 = encode_agg_feature(trn_df,selcols=['app', 'ip','device','os'], groupby=['ip','device','os'])
print(f9.name)
trn_df[f9.name] = f9.values.astype('uint8'); del f9;gc.collect()


#### count #####

print('feature 11 : count by (ip,app)', end='\t')
f11 = encode_agg_feature(trn_df,selcols=['ip','app','channel'],groupby=['ip','app'],aggregator='count')
print(f11.name)
trn_df[f11.name]= f11.values.astype('uint16'); del f11;gc.collect()




print('complete')

feature1 : number of unique "channel" by ip	channel_nunique_ip
feature3 : number of unique "hour" by (ip, day)	hour_nunique_ip_day
feature4 : number of unique "app" by ip	app_nunique_ip
feature7 : number of unique "app" by channel	app_nunique_channel
feature9 : number of unique "app" by (ip,device,os)	app_nunique_ip_device_os
feature 11 : count by (ip,app)	channel_cnt_ip_app
complete


importanta feature 

#### next click

- next click time by ('ip','app','os','device')

In [10]:
print('next click time by(ip,app,os,device)...')
gp = trn_df.groupby(['ip','app','os','device'])
print('shift 1...')

trn_df['shift_1'] = gp.click_time.shift(-1)
cst = pytz.timezone('Asia/Shanghai')
trn_df['shift_1'] = pd.to_datetime(trn_df['shift_1']).dt.tz_localize(pytz.utc).dt.tz_convert(cst) 
trn_df['next_click_dt'] = ((trn_df.shift_1 - trn_df.click_time) / np.timedelta64(1,'s')).fillna(50000).astype('uint16')
print('next_click_dt added')

print('gc...')
trn_df.drop(['shift_1'],axis=1,inplace=True)
gc.collect()
print('complete')

next click time by(ip,app,os,device)...
shift 1...
next_click_dt added
gc...
complete


save temp

In [11]:
store = pd.HDFStore('../input/feat/feat3_partial3.h5')
store['trn_df'] = trn_df
store.close()
    

load 

In [5]:
with pd.HDFStore('../input/feat/feat3_partial3.h5') as store:
    trn_df = store['trn_df']

In [6]:
len_tst = 18790469
len_trn = 143984060 - 18790469

#### mean encode ON TRAINNING ONLY NOT ON TEST ###
- ip mean target
- app mean target 
- channel mean target 
- os mean target
- device mean target

NEED DATA COPY ---> may memory consuming

In [7]:
test_df = trn_df.iloc[len_trn:,]
trn_df  = trn_df.iloc[:len_trn,]

In [11]:
from sklearn.model_selection import StratifiedKFold
print('copy trn_df to make new features')
train_new = trn_df.copy()

cols = ['ip','app','channel','os','device']
for col in cols :
    train_new[col+'_mean_target'] = 0
    
y_tr = trn_df.is_attributed.values.astype(np.int8) # target 
skf = StratifiedKFold(4, random_state=0)
skf.get_n_splits(X=trn_df,y=y_tr)


for fold, (tr_idx, val_idx) in enumerate(skf.split(train_new,y_tr)):
    
    ## generate features 
    
    X_tr ,X_val = trn_df.iloc[tr_idx], trn_df.iloc[val_idx]
    
    print('fold:{} mean encoding...'.format(fold),end='\n\t')
#     print('tr_idx:{}'.format(tr_idx))
    for col in cols:        
        print(col,end='\t')
        means = X_val[col].map(X_tr.groupby(col).is_attributed.mean()) ## map mean encoding in X_tr to X_val
        X_val[col + '_mean_target'] = means.astype('float16')
        
    train_new.iloc[val_idx] = X_val
    print('')    
    del X_tr,X_val;gc.collect()
            
prior = trn_df.is_attributed.mean()
train_new[[col+'_mean_target' for col in cols]] = train_new[[col+'_mean_target' for col in cols]].fillna(prior)

trn_df = train_new
del train_new;gc.collect()
print('complete')

copy trn_df to make new features
fold:0 mean encoding...
	ip	

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


app	channel	os	device	
fold:1 mean encoding...
	ip	app	channel	os	device	
fold:2 mean encoding...
	ip	app	channel	os	device	
fold:3 mean encoding...
	ip	app	channel	os	device	
complete


In [12]:
trn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125193591 entries, 59709852 to 184903442
Data columns (total 23 columns):
app                         uint16
channel                     uint16
click_id                    float64
click_time                  datetime64[ns, Asia/Shanghai]
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               float64
newip                       int32
os                          uint16
channel_nunique_ip          uint8
hour_nunique_ip_day         uint8
app_nunique_ip              uint8
app_nunique_channel         uint8
app_nunique_ip_device_os    uint8
channel_cnt_ip_app          uint16
next_click_dt               uint16
ip_mean_target              float16
app_mean_target             float16
channel_mean_target         float16
os_mean_target              float16
device_mean_target          float16
dtypes: datetime64[ns, Asia/Shanghai](1), float16

___ 

#### mean encode on TEST

In [14]:
cols = ['ip','os','app','device','channel']
selcols = cols + ['is_attributed']

for col in cols:
    print('target_mean col:{}...'.format(col))
    gp = trn_df[selcols].groupby(col)
    test_df[col+'_mean_target'] = test_df[col].map(gp.is_attributed.mean()).astype('float16')
    
prior = trn_df.is_attributed.mean()
test_df[[col+'_mean_target' for col in cols]] = test_df[[col+'_mean_target' for col in cols]].fillna(prior)

gc.collect()
print('complete')

target_mean col:ip...
target_mean col:os...
target_mean col:app...
target_mean col:device...
target_mean col:channel...
complete


In [16]:
test_df.shape

(18790469, 23)

In [17]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 0 to 18790468
Data columns (total 23 columns):
app                         uint16
channel                     uint16
click_id                    float64
click_time                  datetime64[ns, Asia/Shanghai]
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               float64
newip                       int32
os                          uint16
channel_nunique_ip          uint8
hour_nunique_ip_day         uint8
app_nunique_ip              uint8
app_nunique_channel         uint8
app_nunique_ip_device_os    uint8
channel_cnt_ip_app          uint16
next_click_dt               uint16
ip_mean_target              float16
os_mean_target              float16
app_mean_target             float16
device_mean_target          float16
channel_mean_target         float16
dtypes: datetime64[ns, Asia/Shanghai](1), float16(5), floa

___

# Save 



In [18]:
train_df.shape

(37500000, 34)

In [18]:
## day8 train_sets
train_df = trn_df[trn_df.day==8]
## day9 valid sets 
val_df = trn_df[trn_df.day==9]


In [19]:
print("train shape: ", train_df.shape)
print("valid shape: ", val_df.shape)
print("test shape : ", test_df.shape)


train shape:  (62360949, 23)
valid shape:  (62832642, 23)
test shape :  (18790469, 23)


In [11]:
trn_day = 'day8'
val_day = 'day9'
print('../../feat3_trn_{}_val_{}'.format(trn_day,val_day))
store = pd.HDFStore('../input/feat/feat3_trn_{}_val_{}.h5'.format(trn_day,val_day))
store['train_df'] = train_df
store['valid_df'] = val_df
store['test_df'] = test_df
store.close()

../../feat3_trn_day8_val_day9


load

In [2]:
import pandas as pd

In [3]:
with pd.HDFStore('../input/feat/feat3_trn_day8_val_day9.h5') as store:
    train_df = store['train_df']
    val_df   = store['valid_df']
    test_df  = store['test_df']

In [5]:
val_df.newip.value_counts()

0    56367324
1     6465318
Name: newip, dtype: int64

In [6]:
train_df.newip.value_counts()

0    52412023
1     9948926
Name: newip, dtype: int64

In [4]:
train_df['ip_lt126420'] = (train_df.ip<126420).astype('uint8')
test_df['ip_lt126420'] = (test_df.ip<126420).astype('uint8')
val_df['ip_lt126420'] = (val_df.ip<126420).astype('uint8')

In [6]:
train_df.ip_lt126420.value_counts()

1    50165286
0    12195663
Name: ip_lt126420, dtype: int64

In [7]:
val_df.ip_lt126420.value_counts()

1    51222594
0    11610048
Name: ip_lt126420, dtype: int64

In [9]:
test_df.ip_lt126420.value_counts()

1    18790469
Name: ip_lt126420, dtype: int64